In [1]:
import cv2
import numpy as np

src = cv2.imread('../sample/chessboard.png')
gray = cv2.cvtColor(src, cv2.COLOR_BGR2GRAY)

# 예제 9.1 FastFeatureDetector 특징 검출 1
### <span style='background-color:#000a4e'>cv2.FastFeatireDetector</span>
**create(threshold)** 중앙의 화소와 이웃 화소와의 차이의 임계(기본값 10)

In [2]:
fastF = cv2.FastFeatureDetector.create(threshold = 30) # fast feature detector 특징 검출기 객체 생성
# fastF = cv2.FastFeatureDetector_create(threshold= 30)
kp = fastF.detect(gray) # fast feature detector로 특징점 list 구함
print(len(kp))
# for i in kp:
#     print("class id:", i.class_id, "octave:", i.octave, "특징점 좌표:", i.pt, "특징점 반응 세기:", i.response, "의미있는 특징점 이웃의 지름:", i.size, "특징점 방향각:", i.angle)

136


### <span style='background-color:#000a4e'>cv2.drawKeyPoints(src, keypoints, outImage, color)</span>
src에서 검출한 특징점 keypoints를 color 색상으로 outImage에 출력

In [3]:
dst = cv2.drawKeypoints(gray, kp, None, color = (0,0,255))
cv2.imshow('dst', dst)
cv2.waitKey()
cv2.destroyAllWindows()

### <span style='background-color:#000a4e'>setNonmaxSuppresion()</span>
지역극값을 억제하지 않음

In [4]:
fastF.setNonmaxSuppression(False)
kp2 = fastF.detect(gray)
print(len(kp))
dst2 = cv2.drawKeypoints(src, kp2, None, color = (0,0,255))
cv2.imshow('setNonmaxSuppression(False)', dst2)
cv2.waitKey()
cv2.destroyAllWindows()

136


In [5]:
dst3 = src.copy()
points = cv2.KeyPoint_convert(kp)
points = np.int32(points)

for cx, cy in points:
    cv2.circle(dst3, (cx, cy), 3, color = (255,0,0), thickness = 1)
cv2.imshow('list points', dst3)
cv2.waitKey()
cv2.destroyAllWindows()

# 예제 9.2 FastFeatureDetector 특징 검출 2: 특징 정렬, 필터링

In [6]:
fastF = cv2.FastFeatureDetector_create() # threshold (default) 10
kp = fastF.detect(gray)
dst = cv2.drawKeypoints(gray, kp, None, color = (255,0,0))
len(kp)

162

In [7]:
kp = sorted(kp, key=lambda x: x.response, reverse = True) # 특징점 반응 세기 기준으로 내림차순 정렬
# for i in kp[:10]:
#     print(i.pt, i.response)
cv2.drawKeypoints(gray, kp[:10], dst, color = (0,0,255)) # 반응도가 가장 높은 10개 표시
print([i.response for i in kp[:10]])

[253.0, 252.0, 250.0, 250.0, 250.0, 250.0, 249.0, 249.0, 248.0, 243.0]


In [8]:
kp2 = list(filter(lambda f: f.response > 50, kp)) # response가 50 이하인 point 제거
print(len(kp2))
dst2 = cv2.drawKeypoints(gray, kp2, None, color = (0,0,255))

125


In [9]:
def distance(f1, f2):
    x1, y1 = f1.pt
    x2, y2 = f2.pt
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [10]:
def filteringByDistance(kp, distE = 0.5):
    size = len(kp)
    mask = np.arange(1, size + 1).astype(np.bool_)
    for i, f1 in enumerate(kp):
        if not mask[i]:
            continue
        else:
            for j, f2 in enumerate(kp):
                if i == j:
                    continue
                if distance(f1, f2) < distE:
                    mask[j] = False
    print([i for i in mask])
    print("False 개수", list(mask).count(False), "// 필터링 된 pt의 개수")
    np_kp = np.array(kp)
    return list(np_kp[mask])

In [11]:
kp3 = filteringByDistance(kp2, 30)
print("필터링 전 개수", len(kp2), "필터링 이후", len(kp3))
dst3 = cv2.drawKeypoints(gray, kp3, None, color = (0,0,255))

[True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, False, True, True, False, True, True, True, False, True, True, True, False, True, False, True, True, True, True, True, True, True, False, True, True, False, True, True, False, True, True, True, True, False, False, False, False, True, False, False, True, True, False, True, True, False, False, True, False, False, False, True, True, False, False, False, False, True, False, True, False, False, True, True, True, True, False, True, True, False, True, False, True, False, True, True, True, True, True, True, False, True, False, True, True, True, False, True, True, False, True, True, False, True, True, True, False, True, True, False, True, False, True, False, True, True, True, True, True, False, False]
False 개수 42 // 필터링 된 pt의 개수
필터링 전 개수 125 필터링 이후 83


In [12]:
cv2.imshow('top10', dst)
cv2.imshow('response > 50', dst2)
cv2.imshow('filteringByDistance', dst3)
cv2.waitKey()
cv2.destroyAllWindows()